In [18]:
import json
import requests
import pymongo

In [19]:
def get_answers_by_page(topic_id, page_no):
    offset = page_no * 10
    url = "https://www.zhihu.com/api/v4/topics/" + str(topic_id) + "/feeds/essence?include=data%5B%3F(target.type%3Dtopic_sticky_module)%5D.target.data%5B%3F(target.type%3Danswer)%5D.target.content%2Crelationship.is_authorized%2Cis_author%2Cvoting%2Cis_thanked%2Cis_nothelp%3Bdata%5B%3F(target.type%3Dtopic_sticky_module)%5D.target.data%5B%3F(target.type%3Danswer)%5D.target.is_normal%2Ccomment_count%2Cvoteup_count%2Ccontent%2Crelevant_info%2Cexcerpt.author.badge%5B%3F(type%3Dbest_answerer)%5D.topics%3Bdata%5B%3F(target.type%3Dtopic_sticky_module)%5D.target.data%5B%3F(target.type%3Darticle)%5D.target.content%2Cvoteup_count%2Ccomment_count%2Cvoting%2Cauthor.badge%5B%3F(type%3Dbest_answerer)%5D.topics%3Bdata%5B%3F(target.type%3Dtopic_sticky_module)%5D.target.data%5B%3F(target.type%3Dpeople)%5D.target.answer_count%2Carticles_count%2Cgender%2Cfollower_count%2Cis_followed%2Cis_following%2Cbadge%5B%3F(type%3Dbest_answerer)%5D.topics%3Bdata%5B%3F(target.type%3Danswer)%5D.target.annotation_detail%2Ccontent%2Crelationship.is_authorized%2Cis_author%2Cvoting%2Cis_thanked%2Cis_nothelp%3Bdata%5B%3F(target.type%3Danswer)%5D.target.author.badge%5B%3F(type%3Dbest_answerer)%5D.topics%3Bdata%5B%3F(target.type%3Darticle)%5D.target.annotation_detail%2Ccontent%2Cauthor.badge%5B%3F(type%3Dbest_answerer)%5D.topics%3Bdata%5B%3F(target.type%3Dquestion)%5D.target.annotation_detail%2Ccomment_count&limit=10&offset=" + str(page_no)
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36",
    }
    r = requests.get(url, headers=headers)
    content = r.content.decode("utf-8")
    data = json.loads(content)
    is_end = data["paging"]["is_end"]
    items = data["data"]
    client = pymongo.MongoClient()
    db = client["zhihu"]
    if len(items) > 0:
        db.answers.insert_many(items)
        db.saved_topics.insert_one({"topic_id": topic_id, "page_no": page_no})
    return is_end

In [ ]:
def get_answers(topic_id):
    page_no = 0
    client = pymongo.MongoClient()
    db = client["zhihu"]
    while True:
        is_saved = db.saved_topics.find({"topic_id": topic_id, "page_no": page_no}).count()
        if is_saved:
            print("{} {} already saved".format(topic_id, page_no))
            page_no += 1
            continue
        is_end = get_answers_by_page(topic_id, page_no)
        page_no += 1
        if is_end:
            break

In [ ]:
def query():
    client = pymongo.MongoClient()
    db = client.zhihu
    items = db.answers.aggregate([
            {"$addFields": {"answer_len": {"$strLenCP": "$target.content"}}},
            {"$match": {"answer_len": {"$lte": 50}}}
        ])
    answer_ids = []
    for item in items:
        item_type = item["target"]["type"]
        if item_type != "answer":
            continue
        question = item["target"]["question"]["title"]
        answer = item["target"]["content"]
        vote_num = item["target"]["voteup_count"]
        if vote_num < 1000:
            continue
        answer_id = item["target"]["id"]
        if answer_id in answer_ids:
            continue
        url = item["target"]["url"]
        print("=" * 50)
        print("Q: {}\nA: {}\nvote: {}".format(question, answer, vote_num))
        answer_ids.append(answer_id)

In [ ]:
topic_ids = [19554298, 19552330, 19565652, 19580349, 19939299, 19555547, 19594551, 19552832, 19577377, 19552826, 19615452]
for topic_id in topic_ids:
    get_answers(topic_id)

In [31]:
query()

Q: 如果软件正在占领全世界，为什么程序员得不到尊重？
A: 遍身罗绮者，不是养蚕人。 
vote: 8550
Q: 码农们最常说的「谎言」有哪些？
A: //TODO
vote: 2943
Q: 在 GitHub 上保持 365 天全绿是怎样一种体验？
A: <p>曾经保持了200多天全绿，但是冷落了女朋友，一直绿到现在。</p>
vote: 2206
Q: 如何反驳「程序员离开电脑就是废物」这个观点？
A: 不不不，很多程序员在电脑前也是废物。
vote: 2067
Q: 假如有一天所有的人都使用计算机语言说话，会是怎样的场景？
A: <p>hello, world.烫烫烫烫烫烫烫�d}��R�0:�v�?.</p>
vote: 2066
Q: 突然想开一家程序员主题的餐馆，名字就叫程序员的菜，菜名就叫各种语言中的关键字，各位指点一哈，有前途没？
A: 进门一个大大的 hello world <br>招牌菜叫“红烧产品经理”<br>一定会爆满的
vote: 1967
Q: 什么是递归？
A: 「不宜公开讨论的政治内容」的定义和范畴本身也属于「不宜公开讨论的政治内容」
vote: 1707
Q: 编程最基本的术语 “bug” 该怎么翻译？
A: <p>幺蛾子，你的程序又出幺蛾子了。</p>
vote: 1677
Q: 编程的乐趣是什么？
A: 人的成就感来源于两样东西，创造和毁灭。
vote: 1240
Q: 如何反驳「程序员离开电脑就是废物」这个观点？
A: 老实说 跟这种女人都能聊下去 你是不是想上她？
vote: 1188
Q: 作为程序员，你在编程时吃了哪些数学的亏？
A: 看论文时候一个"显然"推了我一下午
vote: 1050
Q: 土豪程序员的设备都有啥？
A: 女朋友。。。
vote: 1018
